## Import Library

In [19]:
from requests import get
from requests.compat import urljoin
from bs4 import BeautifulSoup
from datetime import datetime, timedelta
from tqdm import tqdm
import pandas as pd
import re
import time


## 전처리 함수 정의하기

In [2]:
def preprocessing(d):
    d = d.lower()
    d = re.sub(r'[a-z0-9\-_.]{3,}@[a-z0-9\-_.]{3,}(?:[.]?[a-z]{2})+', ' ', d)
    d = re.sub(r'‘’ⓒ\'\"“”…=□*◆:/_]', ' ', d)
    d = re.sub(r'\s+', ' ', d)
    d = re.sub(r'^\s|\s$', '', d)
    d = re.sub(r'[<*>a-z_="/()]', '', d)
    return d

## Base URL 설정

### 페이지수를 볼 수 있는 네이버 뉴스 주소 (일반 뉴스는 .../main/main.naver?... 주소로 이루어짐)
'https://news.naver.com/main/list.naver?

mode=LSD&

mid=shm&

sid1=100&   -- 카테고리

date={date}& -- 상세카테고리

page={page}' -- 페이지


sid1 (카테고리) : 100(정치), 101(경제), 102(사회), 103(생활문화), 104(세계), 105(IT과학), 106(연예), 107(스포츠), 110(오피니언)

sid2 (상세카테고리) : 


In [3]:
base_url = 'https://news.naver.com/main/list.naver?mode=LSD&mid=shm&sid1=100&date={date}&page={page}'

In [4]:
# 예외 단어 처리 - 재사용하기 위해 컴파일로 미리 저장.
exclude_keywords = re.compile(r'(속보|포토)')

## 자주 쓸 함수 정의

In [20]:
# 기사 콘텐츠 (본문) 수집기 함수 정의
# 작동 안 됨 -> 다음 단계에서 본문 전문 수집. 
# def fetch_article_content(article_url):
#     resp = get(article_url)
#     article_dom = BeautifulSoup(resp.text, 'html.parser')
#     content_tag = article_dom.select_one('div#articleBodyContents')
#     if content_tag:
#         return content_tag.get_text(strip=True)
#     return ''

# 기사 가져오는 함수
def fetch_articles(url):
    resp = get(url)
    dom = BeautifulSoup(resp.text, 'html.parser')
    articles = dom.select('.content ul.type06_headline li')
    article_data_list = []

    # DOM select로 얻어낸 html 반복문 실행하여 데이터 추출
    for article in articles:
        title_tag = article.find('dt', class_=None)
        title_text = title_tag.get_text(strip=True)
       
        # 제목내 예외단어 발견시 추출 건너뛰기 (넘어가기)
        if exclude_keywords.search(title_text):
            continue
    
        # 기사 링크 태그, 주소, 요약 태그, 기자 태그
        link_tag = title_tag.find('a')
        article_url = link_tag['href']
        summary_tag = article.find('span', class_='lede')
        source_tag = article.find('span', class_='writing')
        
        # 기사 콘텐츠 (본문) 수집기
        # content = fetch_article_content(article_url)

        # 기사별 메타데이터 딕셔너리 생성
        article_dict = {
            'title': title_text,
            'link': article_url,
            'summary': summary_tag.get_text(strip=True) if summary_tag else '',
            'source': source_tag.get_text(strip=True) if source_tag else '',
            # 'content': content
        }
        
        # 여러 기사에 대해 메타데이터 작성
        article_data_list.append(article_dict)

    return article_data_list, dom

# 다음 페이지 넘어가기 : 최대 10페이지 전까지 정의
def get_next_page(dom, current_page):
    if current_page < 10:
        next_page_tag = dom.select_one(f'.paging a[href*="page={current_page + 1}"]')
        if next_page_tag:
            return urljoin(base_url, next_page_tag['href'])
    return None

# 일자별 크롤링 함수
def generate_dates(start_date, end_date):
    current_date = start_date
    while current_date <= end_date:
        yield current_date
        current_date += timedelta(days=1)

## 기사 메타데이터 (제목, 링크, 기자, 요약 등) 수집하기

In [21]:
# 기사 수집 시작 / 종료 일자 (ex 2022-05-10: 2022년 5월 10일자)
start_date = datetime.strptime('2024-04-23', '%Y-%m-%d')
end_date = datetime.strptime('2024-04-24', '%Y-%m-%d')

all_articles = []

for single_date in tqdm(generate_dates(start_date, end_date)):
    date_str = single_date.strftime('%Y%m%d')
    page = 1
    url = base_url.format(date=date_str, page=page)
    
    while url and page <= 10:
        article_data_list, dom = fetch_articles(url)
        all_articles.extend(article_data_list)
        url = get_next_page(dom, page)
        page += 1
        time.sleep(2)  # 서버에 부담을 주지 않도록 잠시 대기


## 기사 본문 전문 수집하기

In [48]:
# DataFrame 생성
df1 = pd.DataFrame(all_articles)

In [ ]:
# 기사 링크 리스트(시리즈) 뽑아내기
list(df1['link']) # type : list

In [25]:
# 기사 본문 (전문), 기자 정보 추출하기

article_data_list = []

def fetch_article_data(article_url):
    resp = get(article_url)
    article_dom = BeautifulSoup(resp.text, 'html.parser')
    
    # 기사 제목 추출
    # title_tag = article_dom.select_one('h2#title_area')
    # title = title_tag.get_text(strip=True) if title_tag else ''

    # 기사 본문 추출
    content_tag = article_dom.select_one('article')
    content = preprocessing(content_tag.get_text(strip=True)) if content_tag else ''

    # 언론사 정보 추출
    # source_tag = article_dom.select_one('meta[property="og:article:author"]')
    # source = source_tag['content'] if source_tag else ''
    
    # 기자 정보 추출
    reporter_tag = article_dom.select_one('div.byline span')
    reporter = reporter_tag.get_text(strip=True) if reporter_tag else ''

    article_data = {
        # 'title': title,
        'link': article_url,
        'content': content,
        # 'source': source,
        'reporter': reporter
    }

    return article_data

In [46]:
for url in tqdm(list(df['link'])):
    article_data = fetch_article_data(url)
    article_data_list.append(article_data)

100%|██████████| 193/193 [02:09<00:00,  1.49it/s]


In [47]:
df2 = pd.DataFrame(article_data_list)

## 데이터프레임 합치기

In [ ]:
df = pd.merge(df1, df2)

# 데이터프레임 열 순서 바꾸기
df = df[['source', 'reporter', 'title', 'summary', 'content', 'link']]

# 중복 제거
df.drop_duplicates(subset=None, keep='first', inplace=True, ignore_index=True)

# 열 이름 변경
df.columns = ['언론사', '기자', '제목', '기사요약', '기사전문', '기사링크']

# 데이터프레임 형태 확인
# df.head()

In [67]:
# CSV 파일로 저장
csv_filename = f'뉴스기사_{str(start_date)[:10]}_{str(end_date)[:10]}.csv'
df.to_csv(csv_filename, index=False, encoding='utf-8-sig')

print(f"CSV 파일로 저장되었습니다: {csv_filename}")

CSV 파일로 저장되었습니다: 뉴스기사_2024-04-23_2024-04-24.csv
